In [2]:
%matplotlib inline

import os
import sys
import json
import logging
import warnings
warnings.filterwarnings('ignore')

# 更改为自己的路径,或者直接在环境变量中添加
sys.path.append('/Users/pintec/Documents/repos/genie')

import pandas as pd
import numpy as np
from jinja2 import Template

#python3种reload函数不能直接使用
from imp import reload

from utils3.data_io_utils import *
import utils3.misc_utils as mu
import utils3.summary_statistics as ss
import utils3.metrics as mt
import utils3.feature_selection as fs
import utils3.plotting as pl
import utils3.modeling as ml
import utils3.filing as fl
from functools import reduce
import xgboost as xgb


# 构建取数逻辑（一次性整理成监控所需模样，减少后续工作量）
每个人取值逻辑不同，可根据自身情况调整，该代码可在监控配置中复用，此外表头必须含有apply_id,var_code,create_at字段,
    大家严格按照部署监控时刻的要求撰写PSI监控时刻的SQL取数则不会出错，如果唯一标识不是apply_id，务必将唯一标识字段名修改为apply_id

In [3]:
PSI_SQL = '''
select apply_id, '' as groupby_name, '' as groupby_value, created_at, var_code, var_val from
(
with application as
(
select distinct apply_id
from hive_pro.hdp_data_ods.hdp_ods_m_variable_dict
where dt between '{{start_date}}' and '{{end_date}}'
and productname in ('walletH5Credit','walletAppCredit')
and var_code='channel'
)
,
details as
(
select * from (
select apply_id
        , var_code
        , var_val
        , created_at
        , row_number() over(partition by apply_id, var_code order by created_at desc) as rn
from hive_pro.hdp_data_ods.hdp_ods_m_variable_dict
where dt between '{{start_date}}' and '{{end_date}}'
and productname in ('walletH5Credit','walletAppCredit')	
and var_code in {{var_list}}) where rn=1
)
select application.apply_id
        , details.var_code
        , details.var_val
        , details.created_at
from application inner join details
on application.apply_id = details.apply_id
)'''

# 取出部署时相关文件(xgb的部署文件和评分卡有本质区别，需要文件较多)

In [19]:
DATA_PATH = '/Users/pintec/Nextcloud/Data_Science/分析文件/modeling/new_demo_data/deployment_v6Fraud'

In [20]:
# 部署时刻的模型变量重要性排序,每个人路径不同,根据自己路径修改
feature_importance = pd.read_excel(os.path.join(DATA_PATH,'demoGrid部署文档.xlsx'),sheetname='2_模型变量重要性排序')

In [21]:
# 模型部署时刻decile,每个人路径不同,根据自己路径修改
model_decile = pd.read_excel(os.path.join(DATA_PATH,'demoGrid部署文档.xlsx'),sheetname='3_模型decile')

In [22]:
# 部署时bin_to_label文件,.json文件
bin_to_label = load_data_from_json(DATA_PATH,'v6fraudxgb_bin_to_label.json')

In [23]:
# 部署时selected_rebin_spec文件,.json文件
selected_rebin_spec = load_data_from_json(DATA_PATH,'v6fraudxgb_selected_rebin_spec.json')

In [24]:
# 部署时var_transform_method文件,.json文件
var_transform_method = load_data_from_json(DATA_PATH,'v6fraudxgb_var_transform_method.json')

In [25]:
# 部署时候的booster文件
model_booster = load_data_from_pickle(DATA_PATH,'v6fraudxgb_xgbmodel.pkl')

In [26]:
# 建模时的数据字典
var_dict = pd.read_excel(os.path.join(DATA_PATH,'var_dict.xlsx'))

# 取数

In [27]:
# 将中间层指标以及打分以后的指标一并取出
input_vars = list(set(i for i in feature_importance['中间层指标名称'] if str(i)!='nan'))
output_vars = list(set(i for i in feature_importance['输出打分指标名称'] if str(i)!='nan'))
var_list = tuple(input_vars+output_vars)

In [28]:
# 生成最后的sql,每个人情况不同,根据自身情况而定,但其中一定要有apply_id,var_code,create_at字段
final_sql = Template(PSI_SQL).render(start_date = '2019-01-26',end_date = '2019-01-29',var_list = var_list)

In [29]:
data_B_v3 = presto_read_sql_df(final_sql)

# 生成上线报告相关数据

In [30]:
data_input_vars_missing,data_scored_applys_wrong,model_PSI_summary = ss.xgb_online_score_validity\
(data_B_v3,input_vars,var_dict,var_transform_method,model_booster,model_decile,'mlr_creditscore_xgb_v6fraud')

In [31]:
FINAL_REPORT_PATH = '/Users/pintec/Desktop'

In [32]:
# python3.6的dict是ordered，按照顺序这样定义，之后生成excel的时候会按照顺序穿件sheet
data_dic = {}
# 如果sheet对应的内容是dict，则dict的key会出现在sheet第一列。value会从第二列开始插入
data_dic['01_变量未填充缺失值'] = data_input_vars_missing.reset_index()
data_dic['02_模型打分错误'] = data_scored_applys_wrong.reset_index()
data_dic['03_模型PSI'] = model_PSI_summary

In [33]:
fl.ModelSummary2Excel(FINAL_REPORT_PATH, None, 
                   'XGB_V6Fraud模型分上线验证报告.xlsx', data_dic).run()